# Dowloading Inria-data

In [0]:
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_segmentation_disparity.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_persondetection_train.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_persondetection_test.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_poseestimation_train.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_poseestimation_test.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_negatives.tar
# !wget https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_video_segmentation.tar

--2020-03-22 09:50:28--  https://www.di.ens.fr/willow/research/stereoseg/dataset/inria_stereo_dataset_video_segmentation.tar
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-tar]
Saving to: ‘inria_stereo_dataset_video_segmentation.tar’

inria_stereo_datase     [<=>                 ] 731.17M  9.30MB/s    in 83s     

2020-03-22 09:51:52 (8.84 MB/s) - ‘inria_stereo_dataset_video_segmentation.tar’ saved [766689280]



Decompressing the tar files


In [0]:
# !tar  -xvf inria_stereo_dataset_segmentation.tar
# !tar  -xvf inria_stereo_dataset_persondetection_test.tar
# !tar -xvf inria_stereo_dataset_poseestimation_train.tar
# !tar -xvf inria_stereo_dataset_poseestimation_test.tar
# !tar -xvf inria_stereo_dataset_negatives.tar
# !tar -xf inria_stereo_dataset_video_segmentation.tar

In [0]:
# Download the monocular depth estimation github repo 
!git clone https://github.com/atapour/monocularDepth-Inference.git

In [0]:
# folder_path = 'inria_stereo_dataset/segmentation/frames/'
folder_path = 'inria_stereo_dataset/segmentation/disparity/'

output_path_left = 'left/'
output_path_right = 'right/'
output_path_disparity = 'segmentation_disparity/'

!rm -rf left
!rm -rf right
!mkdir left
!mkdir right
!mkdir segmentation_disparity

Separate left, right images and disparity maps into separate folders

In [0]:
import os

for root, subdirs, files in os.walk(folder_path):
  print(root, subdirs, files)
  for file_ in files:
    if file_.endswith('.png'):
      os.popen('cp {} {}'.format(root+'/'+file_, output_path_disparity+file_))
    if file_.endswith('.jpg'):
      os.popen('cp {} {}'.format(root+'/'+file_, output_path_left+file_))
    elif file_.endswith('.right'):
      os.popen('cp {} {}'.format(root+'/'+file_, output_path_right+file_[:-len('.right')]))

In [0]:
# !cp -r segmentation_left '/content/drive/My Drive/Inria_data'
# !cp -r segmentation_right '/content/drive/My Drive/Inria_data'
# !cp -r checkpoints/ '/content/drive/My Drive/Inria_data/'
# !cp -r left '/content/drive/My Drive/Inria_data/Person/test/left'
# !cp -r right '/content/drive/My Drive/Inria_data/Person/test/right'
# !cp -r left '/content/drive/My Drive/Inria_data/Pose/test/left'
# !cp -r right '/content/drive/My Drive/Inria_data/Pose/test/right'
# !cp -r left '/content/drive/My Drive/Inria_data/Video/left'
# !cp -r right '/content/drive/My Drive/Inria_data/Video/right'

In [0]:
!rm -rf monocular_depth/
!mkdir monocular_depth

In [0]:
# Download the pretrained model
!chmod +x monocularDepth-Inference/download_pretrained_models.sh
!monocularDepth-Inference/download_pretrained_models.sh
!python monocularDepth-Inference/remove_running_stats.py
!python monocularDepth-Inference/run_test.py --data_directory=./left --checkpoints_dir=./checkpoints --results_dir=monocular_depth

python3: can't open file 'monocularDepth-Inference/remove_running_stats.py': [Errno 2] No such file or directory
python3: can't open file 'monocularDepth-Inference/run_test.py': [Errno 2] No such file or directory


In [0]:
!cp -r monocular_depth/ '/content/drive/My Drive/Inria_data/Person/test/'
!cp -r monocular_depth/ '/content/drive/My Drive/Inria_data/Video/'



---

# Evaluating the Generated Depth Maps

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from PIL import Image
import numpy as np
import torchvision
import os

In [4]:
#evaluation 

# return error between two images
# args:
# y_test and y_pred are np.array or list-of-list of same dimensions
# specify metric='rmse' or 'log_rmse' to get that metric's value
def disparity_eval(y_true, y_pred, metric):
    if metric == 'log_rmse':
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    elif metric == 'rmse':
        return np.sqrt(mean_squared_error(y_true, y_pred))

# example
y_true = np.array([[1,0,1],[1,0,0],[0,0,1]])
y_pred_good = np.array([[1,0,1],[1,0,0],[1,0,1]])
y_pred_bad = np.array([[0,1,1],[0,1,0],[1,0,1]])

# pass only one image pair
print(f"closely matched example rmse: {disparity_eval(y_true, y_pred_good, 'rmse')}")
print(f"closely matched example log_rmse: {disparity_eval(y_true, y_pred_good, 'log_rmse')}")
print(f"badly matched example rmse: {disparity_eval(y_true, y_pred_bad, 'rmse')}")
print(f"badly matched example log_rmse: {disparity_eval(y_true, y_pred_bad, 'log_rmse')}")
print(f"exactly matched example rmse: {disparity_eval(y_true, y_true, 'rmse')}")
print(f"exactly matched example rmse_log: {disparity_eval(y_true, y_true, 'rmse_log')}")

closely matched example rmse: 0.3333333333333333
closely matched example log_rmse: 0.23104906018664842
badly matched example rmse: 0.7453559924999298
badly matched example log_rmse: 0.5166414047147861
exactly matched example rmse: 0.0
exactly matched example rmse_log: None


In [15]:
# compute loss b/w disparity-image and depth-map
total_rmse, total_log_rmse = 0, 0
no_imgs = 0
true_folder = '/content/drive/My Drive/Inria_data/Segmentation/disparity/'
pred_folder = '/content/drive/My Drive/Inria_data/Segmentation/monocular_depth/'
# !unzip -q /content/drive/My\ Drive/IDL/Segmentation-20200406T024215Z-001.zip  # pgour
# true_folder = 'Segmentation/disparity/' # for pgour
# pred_folder = 'Segmentation/monocular_depth/' # for pgour


for root, subdirs, files in os.walk(true_folder):
  for true_file in files:
    pred_file = pred_folder + true_file[:-len('.jpg_disparity.png')] + '_depth.png'

    img_true = torchvision.transforms.ToTensor()(Image.open(root+true_file)).squeeze()
    img_pred = torchvision.transforms.ToTensor()(Image.open(pred_file)).squeeze()
    # transform img_pred to be in the format of img_true:
    # - scale img_pred to be from 0-1 instead of 0-65535
    # - transform 0-1 to 1-0 as white/black are used with reversed meanings by deep3D w.r.t inria's labels
    img_pred = 1 - (img_pred.double()/65535)
    
    no_imgs += 1
    total_rmse += disparity_eval(img_true, img_pred,'rmse')
    total_log_rmse += disparity_eval(img_true, img_pred,'log_rmse')

print('**********************')
print('No of images: {}'.format(no_imgs))
print('Avg RMSE:     {}'.format(total_rmse/no_imgs))
print('Avg Log RMSE:     {}'.format(total_log_rmse/no_imgs))
print('**********************')

**********************
No of images: 180
Avg RMSE:     0.3517795048909582
Avg Log RMSE:     0.2270233275741303
**********************
